# Challenge 2

In this challenge we will work on the `Orders` dataset.  

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

1. Create two groups of customers from the dataset: 
    - The first group is **VIP Customers** whose aggregated expenses at your global chain stores are **above the 95th quantile**. 
    - The second group is **Preferred Customers** whose aggregated expenses are **between the 75th and 95th quantile**.
2. Identiy which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.

## Import required libraries

In [1]:
import pandas as pd
import os
import re

## Import data

- Import `Orders` database and save in a variable called `orders`. 
- Print the head of `orders` to overview the data:

Expected output:

>
>|    |   InvoiceNo |   StockCode |   year |   month |   day |   hour | Description                     |   Quantity | InvoiceDate         |   UnitPrice |   CustomerID | Country        |   amount_spent |
|---:|------------:|------------:|-------:|--------:|------:|-------:|:--------------------------------|-----------:|:--------------------|------------:|-------------:|:---------------|---------------:|
|  0 |      546084 |       22741 |   2011 |       3 |     3 |     11 | funky diva pen                  |         48 | 2011-03-09 11:28:00 |        0.85 |        14112 | United Kingdom |          40.8  |
|  1 |      545906 |       22557 |   2011 |       3 |     2 |      9 | plasters in tin vintage paisley |         12 | 2011-03-08 09:23:00 |        1.65 |        15764 | United Kingdom |          19.8  |
|  2 |      539475 |       22176 |   2010 |      12 |     7 |     14 | blue owl soft toy               |          1 | 2010-12-19 14:41:00 |        2.95 |        16686 | United Kingdom |           2.95 |
|  3 |      572562 |       21889 |   2011 |      10 |     2 |      9 | wooden box of dominoes          |         12 | 2011-10-25 09:07:00 |        1.25 |        13481 | United Kingdom |          15    |
|  4 |      549372 |       72741 |   2011 |       4 |     5 |     11 | grand chocolatecandle           |          9 | 2011-04-08 11:28:00 |        1.45 |        14958 | United Kingdom |          13.05 |

In [16]:
order_sample=pd.read_csv('../M01-L13-dataframe_calculations/data/orders_sample.csv')
order_sample.columns=order_sample.columns.str.lower()

## Task 1

### Group values by `CustomerID` and use the sum to aggregate

In [26]:
order_sample

,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,563094,23152,2011,8,4,17,ivory sweetheart wire letter rack,2,2011-08-11 17:35:00,3.75,14525,United Kingdom,7.50
19996,578626,21592,2011,11,4,15,retrospot cigar box matches,24,2011-11-24 15:38:00,1.25,12709,Germany,30.00
19997,561073,23245,2011,7,7,15,set of 3 regency cake tins,16,2011-07-24 15:46:00,4.15,13408,United Kingdom,66.40
19998,576861,20674,2011,11,3,16,green polkadot bowl,12,2011-11-16 16:00:00,1.25,18041,United Kingdom,15.00


In [25]:
order_sample.groupby('customerid')['amount_spent'].sum().reset_index()

,customerid,amount_spent
0,12347,149.90
1,12348,75.36
2,12349,100.09
3,12350,10.20
4,12352,126.48
...,...,...
3321,18276,25.20
3322,18278,29.70
3323,18280,23.60
3324,18283,78.08


- You may notice that the CustomerID column is the index now.
- Use the command `reset_index()` to bring back the CustomerID as a column.
- After that select only the `CustomerID` and `amount_spent` columns.

### Find the quantiles

- Using the `amount_spent` column and apply the `quantile()` function to create two variables:
    - Variable Q95 to store 95th percentile
    - Variable Q75 to store 75th percentile

In [31]:
o=order_sample.groupby('customerid')['amount_spent'].sum().reset_index()
q95=o['amount_spent'].quantile(.95)
q75=o['amount_spent'].quantile(.75)
print(q75,q95)

112.29 385.46000000000004


## Label clientes

- Create a function to label the customers according to their `amount_spent` values and the quantiles.
- Use the Q95 and Q75 variables in your code.

In [34]:
def kind_customer(amount_spent):
    if amount_spent>=q95:
        return ('super customer')
    elif amount_spent>=q75 and amount_spent<q95:
        return ('great customer')
    else:
        return ('radon customer')

- Use `.apply()` to implement the function that you created
- Save in a column called `Label`

In [38]:
order_sample['label']=order_sample['amount_spent'].apply(kind_customer)

## Select the columns `CustomerID` and `Label`
-  Store in a new dataframe

In [41]:
new_order=order_sample[['customerid','label']]
new_order

,customerid,label
0,14112,radon customer
1,15764,radon customer
2,16686,radon customer
3,13481,radon customer
4,14958,radon customer
...,...,...
19995,14525,radon customer
19996,12709,radon customer
19997,13408,radon customer
19998,18041,radon customer


____________________________________

# Taks 2

## Country with more VIP customers

Your will notice that you don't have the Country column in your current dataframe.
- Merge your current dataframe to the `order` dataframe
- Filter the columns `Country`, `Label` and `CustomerID`
- Store this merged dataframe to a variable called `label_country_df`

In [51]:
label_country_df=order_sample
label_country_df
label_country_df=label_country_df[['customerid','label','country']]

We don't want duplicates customres in our count.
- Drop duplicates customers 

In [54]:
label_country_df['customerid'].drop_duplicates(keep='first',inplace=True)

C:\Users\fsarm\AppData\Local\Temp\ipykernel_8344\4021904871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_country_df['customerid'].drop_duplicates(keep='first',inplace=True)


- Create a mask to filter `VIP` clients

In [58]:
mask=label_country_df['label']=='super customer'
vip_customers=label_country_df.loc[mask,:]
vip_customers

,customerid,label,country
555,17450,super customer,United Kingdom
1439,17450,super customer,United Kingdom
1722,12415,super customer,Australia
1940,16013,super customer,United Kingdom
2359,13881,super customer,United Kingdom
...,...,...,...
18088,12590,super customer,Germany
18564,13767,super customer,United Kingdom
18964,18102,super customer,United Kingdom
19451,18102,super customer,United Kingdom


- Now group by country and find the amount of customers 
- Show only the top 10 countries

In [60]:
vip_customers.groupby('country').count().sort_values('label',ascending=False).head(10)

,customerid,label
country,,
United Kingdom,52,52
EIRE,6,6
Australia,4,4
Netherlands,4,4
Germany,3,3


##  Country with more VIP and Preferred customers combined

- Use the `label_country_df` dataframe that you created earlier
- Create a mask to filter `VIP` and `preferred` clients 
- Now group by country and find the amount of customers 
- Show only the top 10 countries

In [17]:
# Your code here